In [1]:
'''This script is a demo of how labels of each mutation types are added'''

import pandas as pd
import re
import numpy as np
from itertools import chain

In [2]:
df_rearrangement = pd.read_excel('report_rearrangement.xlsx')
df_rearrangement.fillna(0,inplace = True)
df_rearrangement

,Type,10A,10B,11A,11B,13A,13B,14A,14B,15D,...,6A,6B,7A,7B,8A,8B,9A,9B,WT,WT_0.25rxn
0,1L1R,250.0,261.0,153.0,154.0,193.0,184.0,176.0,178.0,203.0,...,3.0,0.0,223.0,226.0,138.0,105.0,188.0,170.0,186.0,189.0
1,1LF7LR,2.0,0.0,0.0,0.0,9.0,0.0,2.0,0.0,0.0,...,570.0,579.0,0.0,5.0,330.0,343.0,0.0,1.0,0.0,0.0
2,1R4R5L,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1R7R,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1L7R,1.0,13.0,0.0,1.0,8.0,1.0,3.0,0.0,0.0,...,1.0,0.0,4.0,6.0,27.0,66.0,0.0,1.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,7L7L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167,7LF7LR6RR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168,7RR7LR6RR4RF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,7RR7RF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def deletion_check(protospacer, pattern):
    #check if this may contain delection
    if len(pattern[0]) == 1:
        label = []
        what_to_return = False
        #this may be deletion
        #pair every two elements together for later analysis
        pattern_pairs = [''.join(x) for x in zip(pattern[:-1], pattern[1:])]
        #this is used to concatinate protospacer once the pattern is confirmed
        #protospacer_pairs = ['p'.join(['', *protospacer])]
        #pair every two protospacer number together for later analysis
        protospacer_pairs= [[x,y] for x,y in zip(protospacer[:-1], protospacer[1:])]
        #print(protospacer, pattern, protospacer_pairs,pattern_pairs)
        for i in range(len(pattern_pairs)):
            #everytime before another round in for loop, check if the mutation type is labeled as
            #not meaningful already
            if 'N/A' in label:
                return 'None'
            if pattern_pairs[i] == 'LR' or pattern_pairs[i] == 'LM':
                #if protospacer numbers are the same
                if protospacer_pairs[i][0] == protospacer_pairs[i][1]:
                    #this will be wt label
                    label.append('p'+protospacer_pairs[i][0]+'wt')
                    if what_to_return != 'del' and what_to_return != True:
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0]) < int(protospacer_pairs[i][1]):
                    #this will be deletion label
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'del'
                else:
                    label.append('N/A')
            elif pattern_pairs[i] == 'RL':
                #if the protospacer numbers are continuous
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])-1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'del' and what_to_return != True:
                        what_to_return = 'wt'
                #if protospacer numbers are the same or the numbers are not continuous
                else:
                    label.append('N/A')
            elif pattern_pairs[i] == 'MR':
                #if the protospacer numbers are continuous
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])-1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'del' and what_to_return != True:
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0])+1 < int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'del'
                else:
                    label.append('N/A')
            else:
                #all other combinations are n/a
                label.append('N/A')
        print('this is labal',label)
        if 'N/A' in label:
            return 'None'
        #if deletion present in the label, return only deletion label, hide wt labels
        elif what_to_return == 'del':
            del_labels= [x for x in label if 'del' in x]
            return del_labels
        else:
            return label
    #this may be inversion
    else:
        inv_result = inversion_check(protospacer, pattern)
        return inv_result
    
def inversion_check(protospacer_list, pattern_list):
    label = []
    what_to_return = False
    #pair every two elements together for later analysis
    pattern_pairs = [''.join(x) for x in zip(pattern_list[:-1], pattern_list[1:])]
    #pair every two protospacer number together for later analysis
    protospacer_pairs= [[x,y] for x,y in zip(protospacer_list[:-1], protospacer_list[1:])]
    print(protospacer_list, pattern_list, protospacer_pairs,pattern_pairs)
    for i in range(len(pattern_pairs)):
        #everytime before another round in for loop, check if the mutation type is labeled as
        #not meaningful already
        if 'N/A' in label:
            return 'None'
        if pattern_pairs[i] == 'LFLR' or pattern_pairs[i] == 'RRRF' or pattern_pairs[i] == 'RRMF' \
        or pattern_pairs[i] == 'LFMR' or pattern_pairs[i] == 'MRRF' or pattern_pairs[i] == 'MFLR':
            #if protospacer numbers are NOT the same
            if int(protospacer_pairs[i][0]) < int(protospacer_pairs[i][1]):
                label.append('p'.join(['', *protospacer_pairs[i]])+pattern_pairs[i][0]+pattern_pairs[i][2]+'inv')
                if what_to_return != 'delinv':
                    what_to_return = 'inv'
            elif int(protospacer_pairs[i][0]) > int(protospacer_pairs[i][1]):
                protospacer_pairs[i][0],protospacer_pairs[i][1] =  protospacer_pairs[i][1],protospacer_pairs[i][0] 
                label.append('p'.join(['', *protospacer_pairs[i]])+pattern_pairs[i][0]+pattern_pairs[i][2]+'inv')
                if what_to_return != 'delinv':
                    what_to_return = 'inv'
            else:
                label.append('N/A')
        elif pattern_pairs[i] == 'LFRF'or pattern_pairs[i] == 'RFLF' or pattern_pairs[i]=='LFMF'or pattern_pairs[i]=='MFRF':
            #simplify the pattern
            pattern_sent_for_simplified_check = pattern_pairs[i][::2]
            if pattern_sent_for_simplified_check == 'LR' or pattern_sent_for_simplified_check == 'LM':
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0]) < int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'delinv'
                else:
                    label.append('N/A')
            elif pattern_sent_for_simplified_check == 'RL':
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])-1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                else:
                    label.append('N/A')
            elif pattern_sent_for_simplified_check == 'MR':
                #if the protospacer numbers are continuous
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])-1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0])+1 < int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'delinv'
                else:
                    label.append('N/A')
        elif pattern_pairs[i] == 'LRRR' or pattern_pairs[i] == 'RRLR' or pattern_pairs[i] == 'RRMR':
            pattern_sent_for_simplified_check = pattern_pairs[i][::2]
            if pattern_sent_for_simplified_check == 'LR':
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])+1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                else:
                    label.append('N/A')
            elif pattern_sent_for_simplified_check == 'RL':
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0]) > int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'delinv'
                else:
                    label.append('N/A')
            elif pattern_sent_for_simplified_check == 'RM':
                if int(protospacer_pairs[i][0]) == int(protospacer_pairs[i][1])+1:
                    label.append('p'.join(['', *protospacer_pairs[i]])+'wt')
                    if what_to_return != 'inv' and what_to_return != True and what_to_return != 'delinv' :
                        what_to_return = 'wt'
                elif int(protospacer_pairs[i][0])+1 > int(protospacer_pairs[i][1]):
                    label.append('p'.join(['', *protospacer_pairs[i]])+'del')
                    what_to_return = 'delinv'
                else:
                    label.append('N/A')
        else:
            label.append('N/A')
    if 'N/A' in label:
            return 'None'
    #if inversion present in the label, return only inversion label, hide wt labels
    elif what_to_return == 'inv':
        inv_labels= [x for x in label if 'inv'in x]
        return inv_labels
    elif what_to_return == 'delinv':
        delinv_labels = [x for x in label if 'wt' not in x]
        return delinv_labels
    else:
        return label
def pattern_check(element):
    pattern_rough = r"(\d+)([a-zA-Z]+)"
    match_rough = re.findall(pattern_rough,element)
    protospacer_list = list(''.join(x[0]) for x in match_rough)
    pattern_list = list(''.join(x[1]) for x in match_rough)
    print(match_rough,protospacer_list, pattern_list)
    deletion_output = deletion_check(protospacer_list, pattern_list)
    return deletion_output
    
    
row_name = []
for i in range(df_rearrangement.shape[0]):
    pattern_check_output = pattern_check(df_rearrangement.loc[i][0])
    row_name.append([pattern_check_output,len(pattern_check_output)])
    #print('this is returned output',rowname)
df_rearrangement_label = pd.DataFrame(np.array(row_name),columns = ['mut','num_of_mut_types'])


df_rearrangement_label

[('1', 'L'), ('1', 'R')] ['1', '1'] ['L', 'R']
this is labal ['p1wt']
[('1', 'LF'), ('7', 'LR')] ['1', '7'] ['LF', 'LR']
['1', '7'] ['LF', 'LR'] [['1', '7']] ['LFLR']
[('1', 'R'), ('4', 'R'), ('5', 'L')] ['1', '4', '5'] ['R', 'R', 'L']
[('1', 'R'), ('7', 'R')] ['1', '7'] ['R', 'R']
this is labal ['N/A']
[('1', 'L'), ('7', 'R')] ['1', '7'] ['L', 'R']
this is labal ['p1p7del']
[('1', 'L'), ('6', 'R'), ('7', 'L')] ['1', '6', '7'] ['L', 'R', 'L']
this is labal ['p1p6del', 'p6p7wt']
[('1', 'L'), ('6', 'R')] ['1', '6'] ['L', 'R']
this is labal ['p1p6del']
[('1', 'R'), ('2', 'L')] ['1', '2'] ['R', 'L']
this is labal ['p1p2wt']
[('1', 'L'), ('6', 'R'), ('7', 'R')] ['1', '6', '7'] ['L', 'R', 'R']
this is labal ['p1p6del', 'N/A']
[('1', 'LR'), ('1', 'RF')] ['1', '1'] ['LR', 'RF']
['1', '1'] ['LR', 'RF'] [['1', '1']] ['LRRF']
[('1', 'RR'), ('7', 'RF')] ['1', '7'] ['RR', 'RF']
['1', '7'] ['RR', 'RF'] [['1', '7']] ['RRRF']
[('1', 'LF'), ('5', 'MR')] ['1', '5'] ['LF', 'MR']
['1', '5'] ['LF', 'MR'] [

,mut,num_of_mut_types
0,[p1wt],1
1,[p1p7LLinv],1
2,None,4
3,None,4
4,[p1p7del],1
...,...,...
166,None,4
167,None,4
168,[p4p6RRinv],1
169,None,4


In [9]:
result = pd.concat([df_rearrangement, df_rearrangement_label], axis=1, sort=False)
titles = ['Type','WT','WT_0.25rxn','1A','1A_0.25rxn','1B','2A','2A_0.25rxn','2B','3A','3A_0.25rxn','3B','4B','4C','5A','5B','6A','6B','7A','7B','8A','8B','9A','9B','10A','10B',\
         '11A','11B','13A','13B','14A','14B','15D','16A','16B','17A','17B','18A','18B','19A','19B','20A','20B',\
         '21A','21B','mut','num_of_mut_types']
# output for diversity analysis pipeline
df_rearrangement_with_label = result.reindex(columns=titles)
df_rearrangement_with_label[titles[1:-2]] = df_rearrangement_with_label[titles[1:-2]].astype(int)
df_rearrangement_with_label

,Type,WT,WT_0.25rxn,1A,1A_0.25rxn,1B,2A,2A_0.25rxn,2B,3A,...,18A,18B,19A,19B,20A,20B,21A,21B,mut,num_of_mut_types
0,1L1R,186,189,234,310,276,166,179,262,174,...,274,190,153,164,144,118,2,0,[p1wt],1
1,1LF7LR,0,0,0,0,0,0,0,1,0,...,18,1,1,0,0,6,0,1,[p1p7LLinv],1
2,1R4R5L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,None,4
3,1R7R,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,None,4
4,1L7R,4,5,0,0,0,0,0,0,0,...,1,0,0,0,0,3,0,2,[p1p7del],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,7L7L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,None,4
167,7LF7LR6RR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,None,4
168,7RR7LR6RR4RF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,[p4p6RRinv],1
169,7RR7RF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,None,4


In [10]:
#remove None labelled reads
df_rearrangement_with_label.drop(df_rearrangement_with_label[df_rearrangement_with_label.mut == 'None'].index, inplace=True)
df_rearrangement_with_label.reset_index(inplace = True,drop=True)
df_rearrangement_with_label.to_csv('pattern_check.csv',index=False)
df_rearrangement_with_label

,Type,WT,WT_0.25rxn,1A,1A_0.25rxn,1B,2A,2A_0.25rxn,2B,3A,...,18A,18B,19A,19B,20A,20B,21A,21B,mut,num_of_mut_types
0,1L1R,186,189,234,310,276,166,179,262,174,...,274,190,153,164,144,118,2,0,[p1wt],1
1,1LF7LR,0,0,0,0,0,0,0,1,0,...,18,1,1,0,0,6,0,1,[p1p7LLinv],1
2,1L7R,4,5,0,0,0,0,0,0,0,...,1,0,0,0,0,3,0,2,[p1p7del],1
3,1L6R7L,2,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,3,0,[p1p6del],1
4,1L6R,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p1p6del],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,6RF7LF2LR,3,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,[p2p7LLinv],1
83,6RF7LF3LR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p3p7LLinv],1
84,6RR7RF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p6p7RRinv],1
85,7L7R,39,20,19,9,14,36,26,31,22,...,0,17,38,9,34,40,25,36,[p7wt],1


In [11]:
#if a mutation contains more than one mutation type (num_of_mut_types>0)
#repeat that row
df_repeated = df_rearrangement_with_label.loc[np.repeat(df_rearrangement_with_label.index.values,df_rearrangement_with_label.num_of_mut_types)]
df_repeated.reset_index(drop=True,inplace= True)
df_repeated

,Type,WT,WT_0.25rxn,1A,1A_0.25rxn,1B,2A,2A_0.25rxn,2B,3A,...,18A,18B,19A,19B,20A,20B,21A,21B,mut,num_of_mut_types
0,1L1R,186,189,234,310,276,166,179,262,174,...,274,190,153,164,144,118,2,0,[p1wt],1
1,1LF7LR,0,0,0,0,0,0,0,1,0,...,18,1,1,0,0,6,0,1,[p1p7LLinv],1
2,1L7R,4,5,0,0,0,0,0,0,0,...,1,0,0,0,0,3,0,2,[p1p7del],1
3,1L6R7L,2,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,3,0,[p1p6del],1
4,1L6R,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p1p6del],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,6RF7LF2LR,3,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,[p2p7LLinv],1
103,6RF7LF3LR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p3p7LLinv],1
104,6RR7RF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[p6p7RRinv],1
105,7L7R,39,20,19,9,14,36,26,31,22,...,0,17,38,9,34,40,25,36,[p7wt],1


In [13]:
#if a sequence contain more than one mutation, those mutations are stored in list
#each different mutation is needed to be separated for counting
all_mutationtypes_in_order=list(chain(*df_rearrangement_with_label['mut']))
print(len(all_mutationtypes_in_order))
all_mutationtypes_in_order

107


['p1wt',
 'p1p7LLinv',
 'p1p7del',
 'p1p6del',
 'p1p6del',
 'p1p2wt',
 'p1p7RRinv',
 'p1p5LMinv',
 'p1p4del',
 'p1p5RMinv',
 'p1p4del',
 'p1p4del',
 'p1p4LLinv',
 'p1p5RMinv',
 'p3p5MLinv',
 'p1p5LLinv',
 'p1p5LLinv',
 'p1p5del',
 'p1p7LLinv',
 'p1p7LLinv',
 'p6p5del',
 'p2p6del',
 'p2wt',
 'p2p6del',
 'p2p7del',
 'p2p5del',
 'p2p3LLinv',
 'p2p6RRinv',
 'p2p6RRinv',
 'p2p3RRinv',
 'p2p7LLinv',
 'p2p7RRinv',
 'p2p7LLinv',
 'p2p4RRinv',
 'p2p4del',
 'p2p4del',
 'p2p4RRinv',
 'p2p4LLinv',
 'p2p7LLinv',
 'p2p3del',
 'p3wt',
 'p3p7RRinv',
 'p3p5LMinv',
 'p1p5MRinv',
 'p3p5LMinv',
 'p3p4wt',
 'p3p4del',
 'p3p7LLinv',
 'p3p7LLinv',
 'p5p6del',
 'p4wt',
 'p4p5wt',
 'p5p6del',
 'p4p5wt',
 'p4wt',
 'p4p5wt',
 'p5wt',
 'p4p5wt',
 'p5wt',
 'p5p6wt',
 'p4wt',
 'p4p5wt',
 'p5wt',
 'p5p7del',
 'p4p7del',
 'p4p5del',
 'p1p5LLinv',
 'p4p6RRinv',
 'p2p5LLinv',
 'p5p7del',
 'p5p6del',
 'p4p5wt',
 'p5wt',
 'p5p6wt',
 'p6p7wt',
 'p4p5LMinv',
 'p5p6del',
 'p5p6del',
 'p5p6wt',
 'p6p7wt',
 'p5wt',
 'p5p6wt',

In [14]:
df_separated_mut_label = pd.DataFrame(np.array(all_mutationtypes_in_order),columns = ['sep_mut'])
df_separated_mut_label

,sep_mut
0,p1wt
1,p1p7LLinv
2,p1p7del
3,p1p6del
4,p1p6del
...,...
102,p2p7LLinv
103,p3p7LLinv
104,p6p7RRinv
105,p7wt


In [15]:
df_finalized_rearrangement = pd.concat([df_repeated,df_separated_mut_label],axis=1, sort=False)
df_finalized_rearrangement.drop(['mut','num_of_mut_types'],axis=1,inplace = True)
df_finalized_rearrangement

,Type,WT,WT_0.25rxn,1A,1A_0.25rxn,1B,2A,2A_0.25rxn,2B,3A,...,17B,18A,18B,19A,19B,20A,20B,21A,21B,sep_mut
0,1L1R,186,189,234,310,276,166,179,262,174,...,217,274,190,153,164,144,118,2,0,p1wt
1,1LF7LR,0,0,0,0,0,0,0,1,0,...,1,18,1,1,0,0,6,0,1,p1p7LLinv
2,1L7R,4,5,0,0,0,0,0,0,0,...,19,1,0,0,0,0,3,0,2,p1p7del
3,1L6R7L,2,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,3,0,p1p6del
4,1L6R,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,p1p6del
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,6RF7LF2LR,3,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,p2p7LLinv
103,6RF7LF3LR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,p3p7LLinv
104,6RR7RF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,p6p7RRinv
105,7L7R,39,20,19,9,14,36,26,31,22,...,28,0,17,38,9,34,40,25,36,p7wt


In [17]:

#combine the rearrangement dataframe with the indel dataframe
df_indel= pd.read_csv('indel_count_from_sam_files_ordered_by_mutation_types.csv')
#df_indel.rename(columns={'type': 'Type'},inplace = True)
df_indel['sep_mut'] = 'None'
for i in range(df_indel.shape[0]):
    if 'wt' not in df_indel.loc[i]['Type']:
        df_indel.at[i,'sep_mut'] = df_indel.loc[i][0][0:2]+'indel'
    else:
        df_indel.at[i,'sep_mut'] = df_indel.loc[i][0]+'true'
df_combined = pd.concat([df_finalized_rearrangement, df_indel], sort=False,ignore_index=True)
df_combined

,Type,WT,WT_0.25rxn,1A,1A_0.25rxn,1B,2A,2A_0.25rxn,2B,3A,...,17B,18A,18B,19A,19B,20A,20B,21A,21B,sep_mut
0,1L1R,186,189,234,310,276,166,179,262,174,...,217,274,190,153,164,144,118,2,0,p1wt
1,1LF7LR,0,0,0,0,0,0,0,1,0,...,1,18,1,1,0,0,6,0,1,p1p7LLinv
2,1L7R,4,5,0,0,0,0,0,0,0,...,19,1,0,0,0,0,3,0,2,p1p7del
3,1L6R7L,2,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,3,0,p1p6del
4,1L6R,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,p1p6del
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,p7CATATGTTTT,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,p7indel
285,p7CGCATGGTTT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,p7indel
286,p7CTCATAGTTT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,p7indel
287,p7TACAGTTTTT,0,0,13,12,14,0,0,0,0,...,0,1,0,0,0,0,0,0,0,p7indel


In [224]:
df_combined.to_csv('check_separated_labels.csv',index= False)

In [18]:
df_initiated = pd.DataFrame(np.array([[0, 0, 0,0,0,0], [0, 0, 0,0,0,0], [0, 0, 0,0,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0]]),
                   columns=['del', 'invL','invR', 'indel','unchanged','missingfreq'],index=['p1','p2','p3','p4','p5','p6','p7'],\
                           dtype =np.float)
df_initiated

,del,invL,invR,indel,unchanged,missingfreq
p1,0.0,0.0,0.0,0.0,0.0,0.0
p2,0.0,0.0,0.0,0.0,0.0,0.0
p3,0.0,0.0,0.0,0.0,0.0,0.0
p4,0.0,0.0,0.0,0.0,0.0,0.0
p5,0.0,0.0,0.0,0.0,0.0,0.0
p6,0.0,0.0,0.0,0.0,0.0,0.0
p7,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#output = {'p1':[],'p2':[],'p3':[],'p4':[],'p5':[],'p6':[],'p7':[]}
for i in range(df_combined.shape[0]):
    #print(df_combined.loc[i]['sep_mut'])
    
    if 'wttrue' in df_combined.loc[i]['sep_mut']:
        #print(df_combined.loc[i]['sep_mut'][:2])
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'unchanged'] = df_combined.iloc[i][1:(df_combined.shape[1]-1)].sum()
        missing_plants =(df_combined.loc[i].values == 0).astype(int).sum() / (df_combined.shape[1]-2)
        print(df_combined.loc[i]['sep_mut'][:2],missing_plants)
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'missingfreq'] = missing_plants
    elif 'indel' in df_combined.loc[i]['sep_mut']:
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'indel'] = df_initiated.loc[df_combined.loc[i]['sep_mut'][:2],'indel']+df_combined.iloc[i][1:(df_combined.shape[1]-1)].sum()
    elif 'RRinv' in df_combined.loc[i]['sep_mut'] or  'RMinv' in df_combined.loc[i]['sep_mut'] or 'MRinv' in df_combined.loc[i]['sep_mut']:
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'invR'] = df_initiated.loc[df_combined.loc[i]['sep_mut'][:2],'invR']+df_combined.iloc[i][1:(df_combined.shape[1]-1)].sum()
    elif 'LLinv' in df_combined.loc[i]['sep_mut'] or  'LMinv' in df_combined.loc[i]['sep_mut'] or 'MLinv' in df_combined.loc[i]['sep_mut']:
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'invL'] = df_initiated.loc[df_combined.loc[i]['sep_mut'][:2],'invL']+df_combined.iloc[i][1:(df_combined.shape[1]-1)].sum()
    elif 'del' in df_combined.loc[i]['sep_mut']:
        df_initiated.at[df_combined.loc[i]['sep_mut'][:2],'del'] = df_initiated.loc[df_combined.loc[i]['sep_mut'][:2],'del']+df_combined.iloc[i][1:(df_combined.shape[1]-1)].sum()
    
        

p1 0.09090909090909091
p2 0.3181818181818182
p3 0.5454545454545454
p4 0.09090909090909091
p5 0.6363636363636364
p6 0.7272727272727273
p7 0.1590909090909091


In [20]:
df_initiated

,del,invL,invR,indel,unchanged,missingfreq
p1,1873.0,2548.0,3014.0,15795.0,15676.0,0.090909
p2,4939.0,1234.0,1877.0,3302.0,4603.0,0.318182
p3,2.0,602.0,471.0,6042.0,3010.0,0.545455
p4,979.0,1.0,2.0,1723.0,10499.0,0.090909
p5,6660.0,0.0,0.0,1196.0,2921.0,0.636364
p6,3.0,0.0,2.0,1387.0,2127.0,0.727273
p7,0.0,0.0,0.0,6579.0,12080.0,0.159091


## Equation:
total mutation count+ wt count +  total* missing percentage = total<br />
total = (total mutation count+ wt count)/(1-missing percentage)

In [21]:
df_initiated['total_mutation_and_wt_count'] =df_initiated.loc[:,'del':'unchanged'].sum(axis=1)
df_initiated

,del,invL,invR,indel,unchanged,missingfreq,total_mutation_and_wt_count
p1,1873.0,2548.0,3014.0,15795.0,15676.0,0.090909,38906.0
p2,4939.0,1234.0,1877.0,3302.0,4603.0,0.318182,15955.0
p3,2.0,602.0,471.0,6042.0,3010.0,0.545455,10127.0
p4,979.0,1.0,2.0,1723.0,10499.0,0.090909,13204.0
p5,6660.0,0.0,0.0,1196.0,2921.0,0.636364,10777.0
p6,3.0,0.0,2.0,1387.0,2127.0,0.727273,3519.0
p7,0.0,0.0,0.0,6579.0,12080.0,0.159091,18659.0


In [22]:
df_initiated['total'] =df_initiated['total_mutation_and_wt_count']/(1-df_initiated['missingfreq'])
df_initiated

,del,invL,invR,indel,unchanged,missingfreq,total_mutation_and_wt_count,total
p1,1873.0,2548.0,3014.0,15795.0,15676.0,0.090909,38906.0,42796.600000
p2,4939.0,1234.0,1877.0,3302.0,4603.0,0.318182,15955.0,23400.666667
p3,2.0,602.0,471.0,6042.0,3010.0,0.545455,10127.0,22279.400000
p4,979.0,1.0,2.0,1723.0,10499.0,0.090909,13204.0,14524.400000
p5,6660.0,0.0,0.0,1196.0,2921.0,0.636364,10777.0,29636.750000
p6,3.0,0.0,2.0,1387.0,2127.0,0.727273,3519.0,12903.000000
p7,0.0,0.0,0.0,6579.0,12080.0,0.159091,18659.0,22189.081081


In [23]:
df_initiated['delfreq'] = df_initiated['del'] / df_initiated['total']
df_initiated['invLfreq'] = df_initiated['invL'] / df_initiated['total']
df_initiated['invRfreq'] = df_initiated['invR'] / df_initiated['total']
df_initiated['indelfreq'] = df_initiated['indel'] / df_initiated['total']
df_initiated['unchangedfreq'] = df_initiated['unchanged'] / df_initiated['total']

df_initiated = df_initiated.reindex(sorted(df_initiated.columns), axis=1)

df_initiated

,del,delfreq,indel,indelfreq,invL,invLfreq,invR,invRfreq,missingfreq,total,total_mutation_and_wt_count,unchanged,unchangedfreq
p1,1873.0,0.043765,15795.0,0.369071,2548.0,0.059537,3014.0,0.070426,0.090909,42796.600000,38906.0,15676.0,0.366291
p2,4939.0,0.211062,3302.0,0.141107,1234.0,0.052734,1877.0,0.080211,0.318182,23400.666667,15955.0,4603.0,0.196704
p3,2.0,0.000090,6042.0,0.271192,602.0,0.027020,471.0,0.021141,0.545455,22279.400000,10127.0,3010.0,0.135102
p4,979.0,0.067404,1723.0,0.118628,1.0,0.000069,2.0,0.000138,0.090909,14524.400000,13204.0,10499.0,0.722853
p5,6660.0,0.224721,1196.0,0.040355,0.0,0.000000,0.0,0.000000,0.636364,29636.750000,10777.0,2921.0,0.098560
p6,3.0,0.000233,1387.0,0.107494,0.0,0.000000,2.0,0.000155,0.727273,12903.000000,3519.0,2127.0,0.164845
p7,0.0,0.000000,6579.0,0.296497,0.0,0.000000,0.0,0.000000,0.159091,22189.081081,18659.0,12080.0,0.544412


In [343]:
df_initiated.to_csv('frequencies and counts over all plants.csv')